In [21]:
import ee
import geemap
import eemont
from osgeo import ogr, osr

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [22]:
def shpTOfc(pathToShape):
    import json
    # Open the shapefile, get the layer
    ds = ogr.Open(pathToShape)
    lyr = ds.GetLayer()
    # Create an emtpy feature collection
    fc = ee.FeatureCollection([])
    # Get the feature fields and geometries and add them to the feature collection
    idCount = 1
    for feat in lyr:
        geom = feat.geometry()
        # Automatic coordinate transformation
        from_sr = lyr.GetSpatialRef()
        to_sr = osr.SpatialReference()
        to_sr.ImportFromEPSG(4326)
        cs = osr.CoordinateTransformation(from_sr, to_sr)
        geom.Transform(cs)
        geojson = json.loads(geom.ExportToJson())
        properties = {}
        for i in range(feat.GetFieldCount()):
            field_name = feat.GetFieldDefnRef(i).GetName()
            field_value = feat.GetField(i)
            properties[field_name] = field_value
        ee_feat = ee.Feature(ee.Geometry(geojson), properties)
        # Remove the 'id' property from the feature
        ee_feat = ee_feat.set('id', idCount)
        idCount += 1
        fc = fc.merge(ee.FeatureCollection(ee_feat))
    return fc

def fcTOcsv(json, outPath):
    import pandas as pd
    flag = 0
    featVals = json['features']
    for f in featVals:
        prop = f['properties']
        if flag == 0:
            # Create a pandas data frame with the first element of the values
            out_pd = pd.DataFrame(prop, index=[0])
            flag = 1
        else:
            out_pd = pd.concat([out_pd, pd.DataFrame(prop, index=[0])])
    # Write the pandas dataframe 
    out_pd.to_csv(outPath, index=False)  

In [16]:
Map = geemap.Map(zoom=4)
Map.add_basemap('TERRAIN') # add Google Earth Basemap
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [23]:
###Cargo un shapefile manual
fc = ee.FeatureCollection('projects/ee-mvgonzalez/assets/SELS')
fc

gfw = ee.Image("projects/ee-mvgonzalez/assets/landcoverSA").clip(fc)

ntl = ee.Image("projects/ee-mvgonzalez/assets/ntl_trend").clip(fc)

Map.addLayer(gfw,  {'bands': ['b2'], 'min': 0, 'max': 17, 'palette': ['05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff']}, 'Land Cover Modis')

Map.addLayer(ntl,  {'bands': ['b1'], 'min': -3, 'max': 3, 'palette': ['black', 'white']}, 'NTL')

In [24]:
# Apply the reduction
values = ntl.reduceRegions(fc, ee.Reducer.mean(), 1000).getInfo()
values

csv1 = fcTOcsv(values, 'reducer_ntl.csv')

In [50]:
band_info = gfw.bandNames().getInfo()
band_info


['b1',
 'b2',
 'b3',
 'b4',
 'b5',
 'b6',
 'b7',
 'b8',
 'b9',
 'b10',
 'b11',
 'b12',
 'b13',
 'b14',
 'b15',
 'b16',
 'b17',
 'b18',
 'b19',
 'b20',
 'b21']

In [51]:
# Diccionario para almacenar los resultados
results = {}

# Iterar sobre cada banda de la imagen
for feature in fc.getInfo()['features']:
    # Obtener el ID del feature
    feature_id = feature['id']
    
     # Obtener el nombre del feature
    feature_name = feature['properties']['Name']
    # Diccionario para almacenar los resultados del feature
    feature_results = {}
    
    for band_name in band_info:
        # Reducir la región y contar las categorías de los píxeles en la banda actual
        reducer = ee.Reducer.frequencyHistogram().unweighted()
        histogram = gfw.select([band_name]).reduceRegion( reducer=ee.Reducer.frequencyHistogram().unweighted(),
            geometry=feature['geometry'],
            scale=1000,
            maxPixels=1e13)
         # Obtener el diccionario de frecuencias de píxeles
        histogram_dict = ee.Dictionary(histogram.get(band_name))
        
        # Obtener las categorías y los conteos
        categories = histogram_dict.keys()
        counts = histogram_dict.values()
        
        # Convertir las categorías y los conteos a listas
        categories_list = categories.getInfo()
        counts_list = counts.getInfo()
        
        # Crear un diccionario de categorías y conteos
        pixel_counts = dict(zip(categories_list, counts_list))
        
        # Agregar los resultados de la banda al resultado del feature
        feature_results[band_name] = pixel_counts
    
    # Agregar los resultados del feature al diccionario principal
    results[feature_name] = feature_results


In [55]:
# Imprimir los resultados
print(results)

{'SELS A1': {'b1': {'1': 22292, '10': 16909, '11': 20550, '12': 23, '13': 70, '14': 1, '15': 17498, '16': 106264, '17': 36326, '2': 14817, '3': 4, '4': 2050, '5': 20930, '6': 153, '7': 1502, '8': 7794, '9': 3106}, 'b2': {'1': 22147, '10': 21077, '11': 12253, '12': 17, '13': 69, '15': 16572, '16': 106699, '17': 37169, '2': 14721, '3': 1, '4': 1957, '5': 22589, '6': 142, '7': 1826, '8': 9623, '9': 3427}, 'b3': {'1': 20957, '10': 22211, '11': 10150, '12': 12, '13': 70, '15': 16185, '16': 106810, '17': 37112, '2': 14724, '4': 2154, '5': 24033, '6': 118, '7': 2023, '8': 10081, '9': 3649}, 'b4': {'1': 20297, '10': 22787, '11': 9128, '12': 12, '13': 69, '15': 16838, '16': 106295, '17': 36792, '2': 14820, '4': 2383, '5': 24714, '6': 99, '7': 2111, '8': 10337, '9': 3607}, 'b5': {'1': 19181, '10': 24006, '11': 8662, '12': 11, '13': 68, '15': 17169, '16': 105651, '17': 36613, '2': 14588, '4': 2579, '5': 25094, '6': 129, '7': 2348, '8': 10171, '9': 4019}, 'b6': {'1': 18505, '10': 24061, '11': 8305

In [56]:
df = pd.DataFrame.from_dict(results, orient='index')
# Crear una lista vacía para almacenar los nuevos registros
new_rows = []

# Iterar sobre cada fila del DataFrame
for idx, row in df.iterrows():
    # Iterar sobre las columnas y los valores de la fila actual
    for column, value in row.items():
        # Iterar sobre las claves y los valores del diccionario
        for key, count in value.items():
            # Crear un nuevo registro con el ID del feature, la banda y el valor
            new_row = {'Feature ID': idx, 'Band': column, 'Category': key, 'Count': count}
            # Agregar el nuevo registro a la lista
            new_rows.append(new_row)


In [57]:
# Crear un nuevo DataFrame a partir de la lista de nuevos registros
df_new = pd.DataFrame(new_rows)
df_new

,Feature ID,Band,Category,Count
0,SELS A1,b1,1,22292
1,SELS A1,b1,10,16909
2,SELS A1,b1,11,20550
3,SELS A1,b1,12,23
4,SELS A1,b1,13,70
...,...,...,...,...
4183,SELS E3,b21,5,1352
4184,SELS E3,b21,6,46
4185,SELS E3,b21,7,57
4186,SELS E3,b21,8,146917


In [43]:
df2 = df_new.transpose()

In [58]:
df_new.to_csv('tabla_resultados3.csv', index_label='Feature ID')

In [ ]:
####TIME SERIES PLOT

In [ ]:
series = df_n